In [ ]:
import warnings

import numpy as np
import rasterio
from shapely.geometry import *
from shapely.errors import ShapelyDeprecationWarning

import matplotlib.pyplot as plt
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["axes.facecolor"] = "white"
plt.rcParams["font.family"] = "Arial"

from scrollstats import BendDataset, RASTER_PATHS, calculate_ridge_metrics

In [ ]:
bend_id = "LBR_025"

# Get rasters
dem = rasterio.open(RASTER_PATHS[bend_id]["dem"])
bin_raster = rasterio.open(RASTER_PATHS[bend_id]["bin"])

# Get vectors
bend_ds = BendDataset(bend_id)
ridges = bend_ds.get_ridges(True)
transects = bend_ds.get_transects(True)

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", ShapelyDeprecationWarning)
    rich_transects, itx = calculate_ridge_metrics(transects, bin_raster, dem, ridges)

itx = itx.loc[bend_id]
rich_transects = rich_transects.set_index("transect_id")
chosen_transect = itx.loc["t_016"].sort_values("transect_position")

In [ ]:
for i, row in rich_transects.iterrows():

    fig, ax = plt.subplots(1,1, figsize=(10,3))

    dem_sig = row["dem_signal"]
    bin_sig = row["bin_signal"]
    
    ax.plot(dem_sig, label="DEM")
    ax.plot((bin_sig * (np.nanmax(dem_sig)- np.nanmin(dem_sig)))+ np.nanmin(dem_sig), label="Binary Classification")

    ax.set_title(i)

In [ ]:
rich_transects.columns

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6.5, 3), sharex=True)

dem_sig = rich_transects.loc["t_016", "dem_signal"]
bin_sig = rich_transects.loc["t_016", "bin_signal"]

bin_sig_adj = (bin_sig * (np.nanmax(dem_sig)- np.nanmin(dem_sig))) + np.nanmin(dem_sig)

# Plot DEM and Binary Signal
ax2.plot(dem_sig, color="k", lw=1, label="DEM")
ax2.plot(bin_sig_adj, color="grey", ls="--", lw=1, alpha=0.5, label="Binary Signal")

# Plot transect

all_verts = []
for i, row in chosen_transect.reset_index().iterrows():

    # Determine y height for each substring
    max_y = 78
    y = max_y - 5*(i/len(chosen_transect))

    r0, r1, r2 = row["vertex_indices"]
    ax1.plot([r0, r1, r2], np.ones(3)*y, lw=1, marker=".", markersize=3, markerfacecolor="k", markeredgecolor="k")
    all_verts.append(r0)
    all_verts.append(r1)
    all_verts.append(r2)

    # Annotate first substring
    if i==0:
        ax1.annotate("r0", xy=(r0, y+0.2), horizontalalignment="center")
        ax1.annotate("r1", xy=(r1, y+0.2), horizontalalignment="center")
        ax1.annotate("r2", xy=(r2, y+0.2), horizontalalignment="center")

# Plot complete transect
xs = sorted(set(all_verts))
ys =np.ones(len(xs))*(np.nanmax(dem_sig)+1)
ax2.plot(xs, ys, color="r",lw=1.5, marker=".", markerfacecolor="k", markeredgecolor="k", label="Migration Pathway")



# Remove top and right frames from the bottom plot
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

ax2.set_yticks([65, 70])
ax2.set_ylabel("Elevation [m]")
ax2.set_xlabel("Distance along Migration Pathway [m]")


ax1.set_axis_off()

plt.tight_layout()
plt.savefig("SubStringSampling.png", dpi=300)

In [ ]:
rich_transects.loc["t_016","geometry"].apply

In [ ]:
for i, row in chosen_transect.reset_index().iterrows():
    
    max_y = 75

    y = max_y - 5*(i/len(chosen_transect))
    print(y)

In [ ]:
75 - 5*(1-len(chosen_transect))

In [ ]:

for i, row in chosen_transect.iterrows():

    geom = row["substring_geometry"]
    dem_sig = row["dem_signal_selection"]
    # bin_sig = row["bin_signal"]
    bin_sig = row["bool_mask"]
    pos = row["transect_position"]
    dq_adj = row["swale_dq_adjustment"]

    fig, ax = plt.subplots(1,1, figsize=(6, 3))
    ax.plot(dem_sig, label="DEM")
    ax.plot((bin_sig * (np.nanmax(dem_sig)- np.nanmin(dem_sig)))+ np.nanmin(dem_sig), label="Binary Classification")

    p1, p2, p3 = [Point(p) for p in geom.coords]
    d1 = p1.distance(p2) - dq_adj
    d2 = p2.distance(p3)

    x = [0, d1, d1+d2]
    y = np.ones(3)*np.nanmax(dem_sig)+1
    ax.plot(x, 
            y,
            color="k",
            marker=".",
            markersize=10,
            label = "Transect + Vertices")
    
    ax.annotate(pos, (x[1]+2, y[1]-0.1), verticalalignment="top", horizontalalignment="center", )
    ax.legend()
    ax.set_title("DEM and binary profile at the channel")
    ax.set_ylabel("Elevation [m]")
    ax.set_xlabel("Distance from channel [m]")


## Create Sample Signals for the Confidence table

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(1.5, 4))

# All figures are 6 units long by 3 units high
for ax in axs:
    ax.set_xlim(0.5, 6.5)
    ax.set_ylim(-0.5, 1.5)
    ax.set_xticks([])
    ax.set_yticks([0, 1])
    ax.set_aspect("equal")


# CL 1
x1 = [1.5, 3.5]
y1 = [1, 1]

x2 = [3.5, 5.5]
y2 = [0,0]
axs[0].plot(x1, y1, color="grey", lw=4, alpha=0.5)
axs[0].plot(x2, y2, color="grey", lw=4, alpha=0.5)
axs[0].plot(x1, y1, color="k")
axs[0].plot(x2, y2, color="k")

axs[0].text(3.5, 0.5, "or", horizontalalignment="center", verticalalignment="center")

# CL 2
x = [2.5, 3.5, 3.5, 4.5]
y = [1, 1, 0, 0]

axs[1].plot(x[0:2], y[0:2], color="r", lw=4, alpha=0.5)
axs[1].plot(x[-2:], y[-2:], color="r", lw=4, alpha=0.5)
axs[1].plot(x, y, color="k")

# CL 3
x = [2, 3, 3, 4, 4, 5]
y = [0, 0, 1, 1, 0, 0]
axs[2].plot(x[0:2], y[0:2], color="r", lw=4, alpha=0.5)
axs[2].plot(x[-2:], y[-2:], color="r", lw=4, alpha=0.5)
axs[2].plot(x[2:4], y[2:4], color="b", lw=4, alpha=0.5)
axs[2].plot(x, y, color="k")

# CL 4
x = [1, 2, 2, 3, 3, 4, 4, 5, 5, 6]
y = [1, 1, 0, 0, 1, 1, 0, 0, 1, 1]
axs[3].plot(x[2:4], y[2:4], color="b", lw=4, alpha=0.5)
axs[3].plot(x[-4:-2], y[-4:-2], color="b", lw=4, alpha=0.5)
axs[3].plot(x[4:6], y[4:6], color="b", lw=4, alpha=0.5)
axs[3].plot(x, y, color="k")

plt.savefig("test.png", dpi=300)